### Libraries

In [3]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from unpooling import unpool_layer2x2_batch

from tensorflow.python.framework import ops
from cnn_utils import *

from __future__ import absolute_import
from __future__ import print_function
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
import numpy as np
from keras.applications import imagenet_utils
from keras.utils import np_utils
from keras.layers import Input
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model 

ImportError: No module named google.protobuf

### Functions

In [ ]:
def load_data(labeled_path, unlabeled_path):

	# Find labeled files
	labeled_files = [f for f in listdir(labeled_path) if isfile(join(labeled_path,f))]
	# Find unlabeled files
	unlabeled_files = [f for f in listdir(unlabeled_path) if isfile(join(unlabeled_path,f))]

	# Allocate array for labeled images
	images_labeled = np.empty([len(labeled_files), 360, 480], dtype = 'uint8')

	# Allocate array for unlabeled images
	images_unlabeled = np.empty([len(unlabeled_files), 360, 480, 3], dtype = 'uint8')


	# Reading labeled images
	for n in range(0, len(labeled_files)):
	    images_labeled[n] = cv2.imread(join(labeled_path,labeled_files[n]), 0)

	# Reading unlabeled images
	for n in range(0, len(unlabeled_files)):
	    images_unlabeled[n] = cv2.imread(join(unlabeled_path,unlabeled_files[n]), 1)

	# Preprocess unlabeled images
	images_unlabeled = images_unlabeled.astype('float64')
	X = imagenet_utils.preprocess_input(images_unlabeled)

	# Preprocess labeled images
	n_labeled = len(labeled_files)
	Y = np_utils.to_categorical(images_labeled.flatten(), 12)
	Y = Y.reshape((n_labeled, images_labeled.size / n_labeled, 12))

	return X, Y

In [ ]:
def create_placeholders(n_H0, n_W0, n_C0, n_y): 
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0])
    Y = tf.placeholder(tf.float32, [None, n_y])
    
    return X, Y

In [ ]:
def initialize_parameters():
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [3, 3, 3, 64] Weights, numpy array of shape (f, f, n_C_prev, n_C)
                        W2 : [3, 3, 64, 128]
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    W1 = tf.get_variable("W1", [7, 7, 3, 64], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [7, 7, 64, 64], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable("W3", [7, 7, 64, 128], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W4 = tf.get_variable("W4", [7, 7, 128, 128], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W5 = tf.get_variable("W5", [7, 7, 128, 256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W6 = tf.get_variable("W6", [7, 7, 256, 256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W7 = tf.get_variable("W7", [7, 7, 256, 256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W8 = tf.get_variable("W8", [7, 7, 256, 512], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W9 = tf.get_variable("W9", [7, 7, 512, 512], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W10 = tf.get_variable("W10", [7, 7, 512, 512], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    
    W11 = tf.get_variable("W11", [7, 7, 512, 512], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W12 = tf.get_variable("W12", [7, 7, 512, 512], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W13 = tf.get_variable("W13", [7, 7, 512, 512], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W14 = tf.get_variable("W14", [7, 7, 512, 256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W15 = tf.get_variable("W15", [7, 7, 256, 256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W16 = tf.get_variable("W16", [7, 7, 256, 256], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W17 = tf.get_variable("W17", [7, 7, 256, 128], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W18 = tf.get_variable("W18", [7, 7, 128, 128], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W19 = tf.get_variable("W19", [7, 7, 128, 64], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W20 = tf.get_variable("W20", [7, 7, 64, 64], initializer = tf.contrib.layers.xavier_initializer(seed = 0))

    
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "W4": W4,
                  "W5": W5,
                  "W6": W6,
                  "W7": W7,
                  "W8": W8,
                  "W9": W9,
                  "W10": W10,
                  "W11": W11,
                  "W12": W12,
                  "W13": W13,
                  "W14": W14,
                  "W15": W15,
                  "W16": W16,
                  "W17": W17,
                  "W18": W18,
                  "W19": W19,
                  "W20": W20}
    
    return parameters

In [ ]:
def forward_propagation(X, parameters):
    """
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']
    W6 = parameters['W6']
    W7 = parameters['W7']
    W8 = parameters['W8']
    W9 = parameters['W9']
    W10 = parameters['W10']
    W11 = parameters['W11']
    W12 = parameters['W12']
    W13 = parameters['W13']
    W14 = parameters['W14']
    W15 = parameters['W15']
    W16 = parameters['W16']
    W17 = parameters['W17']
    W18 = parameters['W18']
    W19 = parameters['W19']
    W20 = parameters['W20']
    
    # Encoder 1
    X = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)  
    
    X = tf.nn.conv2d(X, W2, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)  
    X, ind1 = tf.nn.max_pool_with_argmax(X, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
   
    # Encoder 2
    X = tf.nn.conv2d(X, W3, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)

    X = tf.nn.conv2d(X, W4, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)  
    X, ind2 = tf.nn.max_pool_with_argmax(X, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')

    # Encoder 3
    X = tf.nn.conv2d(X, W5, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W6, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W7, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)  
    X, ind3 = tf.nn.max_pool_with_argmax(X, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # Encoder 4
    X = tf.nn.conv2d(X, W8, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W9, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)
    
    X = tf.nn.conv2d(X, W10, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)
    X, ind4 = tf.nn.max_pool_with_argmax(X, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')

    
    # Decoder 1
    X = unpool_layer2x2_batch(X, ind4)
    X = tf.nn.conv2d(X, W11, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W12, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)
    
    X = tf.nn.conv2d(X, W13, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X)
    
    # Decoder 2
    X = unpool_layer2x2_batch(X, ind3)
    X = tf.nn.conv2d(X, W14, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W15, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W16, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    # Decoder 3
    X = unpool_layer2x2_batch(X, ind2)
    X = tf.nn.conv2d(X, W17, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W18, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    # Decoder 4
    X = unpool_layer2x2_batch(X, ind1)
    X = tf.nn.conv2d(X, W19, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.relu(X) 
    
    X = tf.nn.conv2d(X, W20, strides = [1,1,1,1], padding = 'SAME')
    X = tf.contrib.layers.batch_norm(X)
    X = tf.nn.softmax(X) 

    return X
    

In [ ]:
def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    
    return cost

### Model

In [2]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.009,
          epochs = 100, minibatch_size = 64, print_cost = True):
    
    ops.reset_default_graph() # to be able to rerun the model without overwriting tf variables
    num_of_classes = Y_train.shape[1]   
    (m, n_H0, n_W0, n_C0) = X_train.shape
    
    X, Y = create_placeholders(n_H0, n_W0, n_C0, num_of_classes)
    
    parameters = initialize_parameters()
    
    Z3 = forward_propagation(X, parameters)
    
    
    
    
    
    
    
    
    
    
    
    
     # Cost function: Add cost function to tensorflow graph
    cost = compute_cost(Z3, Y)
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                minibatch_cost += temp_cost / num_minibatches
                

            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy, parameters

IndentationError: expected an indented block (<ipython-input-2-200fa8516b0a>, line 4)

### Main

In [ ]:
# Paths
train_labeled_path = './CamVid/trainannot'
train_unlabeled_path = './CamVid/train'
test_labeled_path = './CamVid/testannot'
test_unlabeled_path = './CamVid/test'

# Load dataset
print "Loading training dataset"
X_train, Y_train = load_data(train_labeled_path, train_unlabeled_path)
print "Loading testing dataset"
X_test, Y_test = load_data(test_labeled_path, test_unlabeled_path)

# Define some parameters
input_shape = (360, 480, 3)
minibatch_size = 1
epochs = 100
learning_rate = 0.1

# Train the model
_, _, parameters = model(X_train, Y_train, X_test, Y_test, learning_rate, epochs, minibatch_size)
